In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install bitsandbytes

!pip install accelerate

!pip install -q git+https://github.com/zphang/transformers@c3dc391

!pip install git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08 


!pip install torch torchvision

!pip install -q datasets loralib sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git (to revision e536616888d51b453ed354a6f1e243fecb02ea08) to /tmp/pip-req-build-hd9fnuoq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-hd9fnuoq
  Running command git rev-parse -q --verify 'sha^e536616888d51b453ed354a6f1e243fecb02ea08'
  Running command git fetch -q https://github.com/huggingface/peft.git e536616888d51b453ed354a6f1e243fecb02ea08
  Running command git checkout -q e536616888d51b453ed354a6f1e243fecb02ea08
  Resolved https://github.com/h

In [3]:
!pip install flask flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:

import torch


from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained("yahma/llama-7b-hf")

model = LLaMAForCausalLM.from_pretrained(
    "yahma/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, model_id="/content/drive/MyDrive/healthapp_alpaca7b")




The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'LLaMATokenizer'.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# for cors origin issue
!pip install Flask-Cors


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
!pip install ngrok



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 6.7 MB/s eta 0:00:00
/bin/bash: ngrok: command not found


In [22]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!mv ngrok /usr/local/bin/


--2023-04-19 20:55:34--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  6.46MB/s    in 2.1s    

2023-04-19 20:55:37 (6.46 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [23]:
!ngrok authtoken "<auth-token>"

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [39]:
import re

# Flask app code:

!pip install flask flask-ngrok

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

from flask_cors import CORS

import os

#we add the auth token for ngrok we got after registering
os.environ["NGROK_AUTH_TOKEN"] = "<auth-token"

app = Flask(__name__)
# this line to enable CORS for all origins
CORS(app, resources={r'/inference': {'origins': 'http://localhost:3000'}})

run_with_ngrok(app)

@app.route('/inference', methods=['POST'])
def inference():

    data = request.get_json()

    prompt = data['prompt']

    # Add the instruction and response template
    prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{prompt}
### Response:"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.15,
    )

    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=128,
    )

    results = []

    for s in generation_output.sequences:
        # Remove the <s> and </s> tokens
        result = tokenizer.decode(s).replace("<s>", "").replace("</s>", "").strip()
        results.append(result)

    return jsonify(results)

if __name__ == '__main__':
    app.run()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://cebc-34-81-175-121.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [19/Apr/2023 21:50:40] "OPTIONS /inference HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2023 21:51:06] "POST /inference HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2023 21:51:34] "OPTIONS /inference HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2023 21:51:43] "POST /inference HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2023 22:06:59] "OPTIONS /inference HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2023 22:07:15] "POST /inference HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2023 22:16:59] "OPTIONS /inference HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2023 22:17:06] "POST /inference HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2023 22:17:23] "OPTIONS /inference HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2023 22:17:29] "POST /inference HTTP/1.1" 200 -


In [29]:
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
what is the fat content in 100g apple?
### Response:"""

inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()


generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)
print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=128,
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

Generating...
  <s> Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
what is the fat content in 100g apple?
### Response:
Apples are rich in nutrients and have low levels of fat, making them a healthy choice for snacks or as part of a balanced diet. The amount of fat varies depending on the variety of apples used but generally speaking, one hundred grams (about three-and-a-half ounces) of fresh apples contain less than one gram of total fat.</s>
